In [ ]:
import pandas as pd
import numpy as np
from numpy import array as arr
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from numpy import array as arr
from json import load
from tensorflow import keras
from keras import layers
import pandas as pd
import numpy as np

from tensorflow.keras import Model
from tensorflow.keras.models import load_model

from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.cloud import storage


In [ ]:
np.__version__

'1.19.5'

In [ ]:
tf.__version__

'2.7.0'

In [ ]:
df = pd.read_csv("video_data.csv")
data = df.to_numpy()
np.random.shuffle(data)
ind = {
    "title": 0,
    "label": 2
}

In [ ]:
data_x = np.array([x[ind["title"]] for x in data])
data_y = np.array([x[ind["label"]] for x in data])

In [ ]:
max_length = 20
vocab_size = 800

training_size = int(0.8 * len(data))

raw_training_x = arr(data_x[0:training_size])
training_y = arr(data_y[0:training_size])
raw_testing_x = arr(data_x[training_size:])
testing_y = arr(data_y[training_size:])

tokenizer = Tokenizer(num_words=vocab_size-1,oov_token="<OOV>")
tokenizer.fit_on_texts(raw_training_x)

word_index = tokenizer.word_index

raw_training_x = tokenizer.texts_to_sequences(raw_training_x)
training_x = pad_sequences(raw_training_x, maxlen=max_length, truncating="post")

raw_testing_x = tokenizer.texts_to_sequences(raw_testing_x)
testing_x = pad_sequences(raw_testing_x, maxlen=max_length, padding="post")

In [ ]:
json.dump(tokenizer.to_json(), open("token.json","w"))

In [ ]:
tokenizer.to_json()

'{"class_name": "Tokenizer", "config": {"num_words": 799, "filters": "!\\"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n", "lower": true, "split": " ", "char_level": false, "oov_token": "<OOV>", "document_count": 1603, "word_counts": "{\\"something\\": 39, \\"about\\": 55, \\"mega\\": 4, \\"man\\": 15, \\"x\\": 7, \\"animated\\": 26, \\"loud\\": 33, \\"sound\\": 34, \\"flashing\\": 10, \\"light\\": 11, \\"warning\\": 32, \\"\\\\ud83c\\\\udf4b\\\\ud83d\\\\udd2b\\": 1, \\"\\\\ud83e\\\\udd16\\": 1, \\"tight\\": 2, \\"core\\": 5, \\"workout\\": 62, \\"15\\": 14, \\"days\\": 6, \\"plank\\": 3, \\"challenge\\": 33, \\"these\\": 5, \\"glasses\\": 1, \\"cured\\": 1, \\"my\\": 68, \\"colorblindness\\": 1, \\"monster\\": 7, \\"hunter\\": 5, \\"world\\": 45, \\"part\\": 29, \\"3\\": 29, \\"\\\\ud83e\\\\udd85\\": 1, \\"\\\\ud83d\\\\udc46\\": 2, \\"greasy\\": 1, \\"cocaine\\": 2, \\"national\\": 33, \\"geographic\\": 33, \\"can\\": 17, \\"you\\": 76, \\"survive\\": 2, \\"quicksand\\": 1, \\"i\\": 67, \\"di

In [ ]:
model = keras.Sequential([
                          layers.Embedding(vocab_size, 64),
                          layers.Bidirectional(layers.LSTM(64)),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.save('./projecte.h5')

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          51200     
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 125,569
Trainable params: 125,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 30
nlp_model = model.fit(training_x, training_y, epochs=num_epochs, validation_data=(testing_x, testing_y), verbose=2)

Epoch 1/30
51/51 - 6s - loss: 0.6150 - accuracy: 0.6438 - val_loss: 0.4854 - val_accuracy: 0.6708 - 6s/epoch - 117ms/step
Epoch 2/30
51/51 - 1s - loss: 0.1951 - accuracy: 0.9295 - val_loss: 0.5981 - val_accuracy: 0.6484 - 1s/epoch - 24ms/step
Epoch 3/30
51/51 - 1s - loss: 0.0984 - accuracy: 0.9607 - val_loss: 0.4166 - val_accuracy: 0.7631 - 1s/epoch - 23ms/step
Epoch 4/30
51/51 - 1s - loss: 0.0703 - accuracy: 0.9713 - val_loss: 0.5631 - val_accuracy: 0.7032 - 1s/epoch - 24ms/step
Epoch 5/30
51/51 - 1s - loss: 0.0475 - accuracy: 0.9794 - val_loss: 0.6062 - val_accuracy: 0.6933 - 1s/epoch - 23ms/step
Epoch 6/30
51/51 - 1s - loss: 0.0488 - accuracy: 0.9794 - val_loss: 0.7540 - val_accuracy: 0.6384 - 1s/epoch - 23ms/step
Epoch 7/30
51/51 - 1s - loss: 0.0326 - accuracy: 0.9875 - val_loss: 0.8210 - val_accuracy: 0.6509 - 1s/epoch - 23ms/step
Epoch 8/30
51/51 - 1s - loss: 0.0543 - accuracy: 0.9807 - val_loss: 1.3948 - val_accuracy: 0.6110 - 1s/epoch - 24ms/step
Epoch 9/30
51/51 - 1s - loss: 0

In [ ]:
model.save_weights('checkpoints/weights.ckpt')

In [ ]:
from numpy import array as arr
from json import load

from tensorflow.keras import Model
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras import layers

from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size=800
max_length=20

json_path='token.json'
weights_path='checkpoints/weights.ckpt'
model_path='projecte.h5'

with open(json_path) as json_file:
    tokenizer = tokenizer_from_json(load((json_file)))

print(pad_sequences(tokenizer.texts_to_sequences(arr(["Nail polish is clickbait"])),
                     maxlen=max_length, truncating="post"))

mod = keras.Sequential([
                          layers.Embedding(vocab_size, 64),
                          layers.Bidirectional(layers.LSTM(64)),
                          layers.Dense(64, activation='relu'),
                          layers.Dense(1, activation='sigmoid')
])

mod.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
mod.load_weights(weights_path)

#model = keras.models.load_model('projecte.h5')

print(tokenizer.num_words)

def isProductive(title):
  proc = pad_sequences(tokenizer.texts_to_sequences(arr([title])),
                     maxlen=max_length, truncating="post")
  
  predic = mod(arr(proc))
  predic = predic.numpy()[0][0]

  return (predic)

isProductive('diy french polynail gels for beginners')

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 21  1 32  1]]
799


1.0

In [ ]:
lay = model.layers.copy()
print(lay[0].input_dim)
print(tokenizer.num_words+1)
#[0]print("Weights: "+model.layers.)

800
800


In [ ]:
isProductive("Learn how to code in 15 minutes")

1.0

In [ ]:
tokenizer.texts_to_sequences(["Nail polish is shit"])

[[27, 1, 35, 1]]

In [ ]:
tokenizer_from_json(load(open("token.json"))).texts_to_sequences(["nail polish is shit"])

[[27, 1, 35, 1]]

In [ ]:
isProductive("Learn how to code in 15 mins")

0.9999574